In [ ]:
!python src/WikiSplitter.py split-wiki dump.xml


In [ ]:
%pip install -U pip setuptools wheel
%pip install -U spacy


In [ ]:
!python -m spacy download en_core_web_sm


In [13]:
import spacy
import json

nlp = spacy.load("en_core_web_sm")


In [83]:
!node run.js 100


--- Arbroath Abbey
--- Abano Terme
--- Aeclanum
--- Aegadian Islands
--- Ajaigarh
--- Ajanta Caves
--- Ajmer
--- Afyonkarahisar
--- Abdera, Spain
--- Aberavon (UK Parliament constituency)
--- Abercarn
--- Achiltibuie
--- Azores
--- Outback
--- Ansbach
--- Aveiro, Portugal
--- Bosporus
--- Cheddar, Somerset
--- Chojnów
--- Actium
--- Aberdeen
error creating shortDescription
--- Abadeh
--- Abae
--- Abakan
--- Aswan
--- Arapaoa Island
--- Aberdour
--- Abersychan
--- Andalusia
--- Azad Kashmir
--- American Registry for Internet Numbers
--- Akihabara
--- Belfast
--- Cayman Islands
--- London Borough of Croydon
--- Cholistan Desert
error creating shortDescription
--- The Maritimes
--- Carson City, Nevada
--- Caribbean Sea
--- Concord, Michigan
--- California Department of Transportation
--- Degree Confluence Project
--- Dimona
--- Davis, California
error creating shortDescription
--- Gestapo
--- Azincourt
--- Achill Island
--- Ames, Iowa
--- Abensberg
--- Andes
--- Anazarbus
--- Anadyr (rive

In [84]:
with open('parsed_articles.json') as file:
    parsed_articles = json.load(file)


In [85]:
def clean_str(string: str):
  return string.replace(
                '\n', ' ').replace('  ', ' ').strip()


In [86]:
articles = []
for article in parsed_articles:
    doc = nlp(article['history'])
    sentences = list(doc.sents)
    sentences_with_dates = []

    for ent in doc.ents:
        if ent.label_ == "DATE":
            current_sentence = ent.sent
            current_index = sentences.index(current_sentence)
            current_sentence_text = clean_str(current_sentence.text)

            # Hinzufügen des vorherigen Satzes, wenn vorhanden und noch nicht in der Liste
            # if current_index > 0 and sentences[current_index - 1].text not in sentences_with_dates:
                # sentences_with_dates.append(sentences[current_index - 1].text)

            # Hinzufügen des aktuellen Satzes, wenn noch nicht in der Liste
            if current_sentence_text not in sentences_with_dates:
                sentences_with_dates.append(current_sentence_text)

            # Hinzufügen des nachfolgenden Satzes, wenn vorhanden und noch nicht in der Liste
            # if current_index < len(sentences) - 1 and sentences[current_index + 1].text not in sentences_with_dates:
                # sentences_with_dates.append(sentences[current_index + 1].text)
        
    article['sentencesWithDates'] = ' '.join(sentences_with_dates)
    articles.append(article)


In [92]:
len(articles)


255

In [88]:
import re


def remove_first_parentheses_in_first_sentence(text):
    sentences = re.split(r'(?<=[.!?])\s+', text, maxsplit=1)
    first_sentence = sentences[0]
    remaining_text = sentences[1] if len(sentences) > 1 else ''
    cleaned_first_sentence = re.sub(r'\(.*?\)', '', first_sentence, count=1)
    return cleaned_first_sentence + ' ' + remaining_text


In [89]:
cleaned_articles = []
for article in articles:
    if article['sentencesWithDates'] == '':
        continue
    if 'shortDescription' in article:
        article['shortDescription'] = remove_first_parentheses_in_first_sentence(
            article['shortDescription'])
    if 'description' in article:
        article['description'] = remove_first_parentheses_in_first_sentence(
            article['description'])
    if article['sentencesWithDates'] == '':
        continue
    cleaned_articles.append(article)


In [93]:
import requests


def get_wikipedia_pageviews_2022(title):
    """
    Ermittelt die Gesamtzahl der Seitenaufrufe für einen Wikipedia-Artikel im Jahr 2022.
    
    :param title: Titel des Wikipedia-Artikels.
    :return: Anzahl der Seitenaufrufe oder eine Fehlermeldung.
    """
    # URL für die Wikipedia API, die Seitenaufrufe für das Jahr 2022 abruft
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/user/{title}/daily/2022010100/2022123100"

    try:
        headers = {
          'User-Agent': 'MeinPythonSkript/1.0 (meineemail@example.com)'
        }
        response = requests.get(url, headers=headers)

        response.raise_for_status()
        data = response.json()

        total_views = sum(item['views'] for item in data['items'])
        return total_views
    except requests.RequestException as e:
        return f"Fehler bei der Anfrage: {e}"
    except KeyError:
        return "Fehler bei der Verarbeitung der Daten."


In [94]:
cleaned_articles_with_clicks = []
for article in cleaned_articles:
    if 'title' in article:
        article['clicks'] = get_wikipedia_pageviews_2022(article['title'])
        print(f"Seitenaufrufe für {article['title']}: {article['clicks']}")
    else:
        article['clicks'] = None
        print(f"Keine Seitenaufrufe für diesen {article['title']} verfügbar.")
    cleaned_articles_with_clicks.append(article)


Seitenaufrufe für Alberta: 1020808
Seitenaufrufe für Ankara: 640512
Seitenaufrufe für Aberdeenshire: 198238
Seitenaufrufe für Azincourt: 41905
Seitenaufrufe für Achill Island: 102543
Seitenaufrufe für Anguilla: 529462
Seitenaufrufe für Ashmore and Cartier Islands: 72160
Seitenaufrufe für Ames, Iowa: 169544
Seitenaufrufe für Abensberg: 5275
Seitenaufrufe für Andes: 521063
Seitenaufrufe für Anazarbus: 7764
Seitenaufrufe für Anadyr (river): 3459
Seitenaufrufe für Ambracia: 9280
Seitenaufrufe für Abydos, Egypt: 95347
Seitenaufrufe für Aberdeen: 437607
Seitenaufrufe für Actium: 26777
Seitenaufrufe für Abadan, Iran: 38464
Seitenaufrufe für Adobe Inc.: 610329
Seitenaufrufe für Austin, Texas: 1397545
Seitenaufrufe für Crandall University: 13363
Seitenaufrufe für Acadia University: 35829
Seitenaufrufe für Annapolis Valley: 21087
Seitenaufrufe für AFC Ajax: 1923325
Seitenaufrufe für Area 51: 1727047
Seitenaufrufe für Albion, Michigan: 15618
Seitenaufrufe für Alicante: 294728
Seitenaufrufe für Ab

In [20]:
with open('articles_with_dates.json', 'w') as file:
    json.dump(cleaned_articles, file, ensure_ascii=False,)


In [ ]:
for article in articles:
    print(
        article['title'], len(article['history']), len(article['sentencesWithDates']))
    print('------------------')


In [ ]:
%pip install openai python-dotenv


In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(
  api_key=os.getenv("OPENAI_API_KEY")
)


In [11]:
def get_messages(text):
  return [
    {
      "role": 'system',
      "content": 'Your task is to extract dates and related event information from the provided text.',
    },
    {
      "role": 'user',
      "content": f'Extract dates from this text: {text}',
    },
  ]
  

In [35]:
functions = [
    {
        "name": 'extract_dates',
        "description": 'Extracts dates and related information from text.',
        "parameters": {
            "type": 'object',
            "properties": {
                "text": {
                    "type": 'string',
                    "description": "The text from which to extract dates and event information."
                }
            },
            "required": ["text"]
        },
        "return": {
            "type": 'array',
            "items": {
                "type": 'object',
                "properties": {
                    "shortTitle": {
                        "type": 'string',
                        "description": "Title of the event. Not longer than 3 words."
                    },
                    "description": {
                        "type": 'string',
                        "description": 'Description of the event. Not longer than 3 sentences.'
                    },
                    "dates": {
                        "type": 'object',
                        "properties": {
                            "date": {
                                "type": 'string',
                                "description": 'Date of the event in YYYY-MM-DD format.'
                            },
                            "scale": {
                                "type": 'string',
                                "description": 'Scale of the event. Possible values: day, month, year, decade, century, millennium.',
                                "enum": ['day', 'month', 'year', 'decade', 'century', 'millennium']
                            }
                        }
                    }
                }
            }
        }
    }
]


In [36]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=get_messages(cleaned_articles[0]['sentencesWithDates']),
    functions=functions,
)
response


ChatCompletion(id='chatcmpl-8PbfL2kYxFMp6in0U36m5MLkOmNWM', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"text":"Paleo-Indians arrived in Alberta at least 10,000 years ago, toward the end of the last ice age. The first Europeans to visit Alberta were French Canadians during the late 18th century, working as fur traders. This area was granted by Charles II of England to the Hudson\'s Bay Company (HBC) in 1670, and rival fur trading companies were not allowed to trade in it. Other North American fur traders formed the North West Company (NWC) of Montreal to compete with the HBC in 1779. Peter Pond built Fort Athabasca on Lac la Biche in 1778. Roderick Mackenzie built Fort Chipewyan on Lake Athabasca ten years later in 1788. The extreme southernmost portion of Alberta was part of the French (and Spanish) territory of Louisiana and was sold to the United States in 1803. In the Tr

In [61]:
response.choices[0].message.content


In [71]:
import openai
import requests

openai.api_key = os.getenv("OPENAI_API_KEY")


In [68]:
GPT_MODEL = "gpt-3.5-turbo-1106"


In [69]:
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [65]:
tools = [
    {
       "type": "function",
      "function": {
        "name": 'extract_dates',
        "description": 'Extracts dates and related information from text.',
        "parameters": {
            "type": 'object',
            "properties": {
                "name": "date",
                "type": 'array',
                "items": {
                    "type": 'object',
                    "properties": {
                        "shortTitle": {
                            "type": 'string',
                            "description": "Title of the event. Not longer than 3 words."
                        },
                        "description": {
                            "type": 'string',
                            "description": 'Description of the event. Not longer than 3 sentences.'
                        },
                        "dates": {
                            "type": 'object',
                            "properties": {
                                "date": {
                                    "type": 'string',
                                    "description": 'Date of the event in YYYY-MM-DD format.'
                                },
                                "scale": {
                                    "type": 'string',
                                    "description": 'Scale of the event. Possible values: day, month, year, decade, century, millennium.',
                                    "enum": ['day', 'month', 'year', 'decade', 'century', 'millennium']
                                }
                            }
                        }
                    }
                }
            },
            "required": ["date"]
        },
    }}
]


In [72]:
chat_response = chat_completion_request(
    get_messages(cleaned_articles[0]['sentencesWithDates']
      ), tools=tools
)


In [76]:
assistant_message = chat_response.json()


In [77]:
assistant_message


{'error': {'message': "Invalid schema for function 'extract_dates': 'date' is not of type 'object', 'boolean'",
  'type': 'invalid_request_error',
  'param': None,
  'code': None}}